In [29]:
import os
import time
import re

width, height, depth = 64, 64, 44
nlabels = 3
num_steps = 5
#method = 1

folder_root = '../'
file_name = 'E13260_run8_3D_wholeBrain_allConds.mat'



def model(width, height, depth, nlabels, num_steps, folder_root, file_name, batchsize, learning_rate, method=1):
    """
    Design three methods to train:
    method 1: Seperate one run as training set and validation set
    method 2: one run as training set, second run as validation set
    method 3: Seperate all data as training set and validation set

    Argument:
        width, height, depth -- the size of one volume
        nlabels -- the number of labels(here is 3 types of brain activity: negative, positive and rest)
        num_steps -- the number of iterations in training process
        method -- type of method
        folder_root -- root which stores files
        file_name -- can be a name of mat file in method one; a dictionary stored two runs of a person; a root stored all files

    Returns
        the loss and accuracy was stored in a txt file
    """
    if method == 1:
        root = folder_root + file_name
        #result_saved_root = folder_root + 'result/' + "method_" + str(method) + "/" + os.path.basename(root)[0:11] + '.txt'
        #hxf 4/28/2018
        #result_saved_root = folder_root + 'result/' + "method_" + str(method) + "/" + os.path.basename(root)[0:11] + "_lr" + str(learning_rate) + "_n" + str(num_steps) + '.txt'
        #dxz 8/9/2018 Use regular expression to extract the object name and run number from the file name, e.g: E14163, run7
        result_saved_root = folder_root + 'result/' + "method_" + str(method) + "/" + re.findall('E[0-9]+',os.path.basename(root))[0]+'_'+re.findall('run[0-9]+',os.path.basename(root))[0] + "_lr" + str(learning_rate) + "_n" + str(num_steps) + '.txt'
        # get data set
        train_X, train_y, val_X, val_y, test_X, test_y = data_preprocessing(root)

    elif method == 2:
        #result_saved_root = folder_root + 'result/' + "method_" + str(method) + "/" + file_name['train'][0:6] + '.txt'
        #hxf 4/28/2018
        #result_saved_root = folder_root + 'result/' + "method_" + str(method) + "/" + file_name['train'][0:6]  + "_lr" + str(learning_rate) + "_n" + str(num_steps) + '.txt'
        #didn't work, TypeError: string indices must be integers

        '''
        #still didn't work in the MRI unit cluster
        #convert from string to dictionary
        #refer to https://stackoverflow.com/questions/988228/convert-a-string-representation-of-a-dictionary-to-a-dictionary
        import json
        #s = "{'muffin' : 'lolz', 'foo' : 'kitty'}"
        json_acceptable_string = file_name.replace("'", "\"")
        file_name = json.loads(json_acceptable_string)
        '''

        #goal: use previous run for training the model, new run's data for testing

        #option#1: run 7 is used for training, run 8 is used for validation, run 16 is used for testing
        # so fixed here!
        # file_name = {"train": 'E13260_run8_3D_wholeBrain_allConds.mat', "val": 'E13260_run9_3D_wholeBrain_allConds.mat'}
        file_name = {"train": 'E14163_run7_3D_wholeBrain_allConds.mat', "val": 'E14163_run8_3D_wholeBrain_allConds.mat', "test": 'E14163_run16_3D_wholeBrain_allConds.mat'}

        result_saved_root = folder_root + 'result/' + "method_" + str(method) + "/" + file_name['train'][0:6] + "_lr" + str(learning_rate) + "_n" + str(num_steps) + '.txt'
        train_root = folder_root + file_name['train']
        val_root = folder_root + file_name['val']
        train_X, train_y = data_preprocessing(train_root, 'True')
        val_X, val_y = data_preprocessing(val_root, 'True')
        # hxf 4/30/2018
        test_root = folder_root + file_name['test']
        test_X, test_y = data_preprocessing(test_root, 'True')

        # option#2: mixed run 7 & 8 for training & validation, run 16 is used for testing

        # option#3: mixed run 7 & 8 for training & validation, use part of run 16 as for updating the model, rest of run 16 is used for testing

        print(train_X.shape)
        print(train_y.shape)
        print(val_X.shape)
        print(val_y.shape)

    elif method == 3:
        #i.e., file_name is useless
        #hxf 4/28/2018
        result_saved_root = folder_root + 'result/' + "method_" + str(method) + "/" + 'allSubjects' + "_lr" + str(learning_rate) + "_n" + str(num_steps) + '.txt'
        #result_saved_root = folder_root + 'result/' + "method_" + str(method) + "/" + 'whole' +'_lr_'+str(learning_rate)+ '.txt'

        #hxf 4/28/2018, if input is only one .mat file
        #tempFilename, tempFileExt = os.path.splitext(file_name)
        #result_saved_root = folder_root + 'result/' + "method_" + str(method) + "/" + tempFilename +'_lr_'+str(learning_rate)+ '.txt'

        # added testing set
        train_X = np.empty((0, width, height, depth))
        val_X = np.empty((0, width, height, depth))
        train_y = np.empty((3, 0))
        val_y = np.empty((3, 0))
        test_X = np.empty((0, width, height, depth))
        test_y = np.empty((3, 0))
        for name in os.listdir(folder_root):
            if os.path.basename(name)[-3:] == 'mat':
                root = folder_root + name
                train_X_batch, train_y_batch, val_X_batch, val_y_batch, test_X_batch, test_y_batch = data_preprocessing(
                    root)
                train_X = np.concatenate((train_X, train_X_batch), axis=0)
                val_X = np.concatenate((val_X, val_X_batch), axis=0)
                train_y = np.concatenate((train_y, train_y_batch), axis=1)
                val_y = np.concatenate((val_y, val_y_batch), axis=1)
                test_X = np.concatenate((test_X, test_X_batch), axis=0)
                test_y = np.concatenate((test_y, test_y_batch), axis=1)
                
        assert (train_X.shape[0] == train_y.shape[1])
        assert (val_X.shape[0] == val_y.shape[1])
        assert (test_X.shape[0] == test_y.shape[1])

        '''
        train_X = np.empty((0,width,height,depth))
        val_X = np.empty((0,width,height,depth))
        train_y = np.empty((3,0))
        val_y = np.empty((3,0))
        for name in os.listdir(folder_root):
            if os.path.basename(name)[-3:] == 'mat':
                root = folder_root + name
                train_X_batch, train_y_batch, val_X_batch, val_y_batch = data_preprocessing(root)
                train_X = np.concatenate((train_X,train_X_batch), axis = 0)
                val_X = np.concatenate((val_X,val_X_batch), axis = 0)
                train_y = np.concatenate((train_y,train_y_batch), axis = 1)
                val_y = np.concatenate((val_y,val_y_batch), axis = 1)
        assert(train_X.shape[0]==train_y.shape[1])
        assert(val_X.shape[0]==val_y.shape[1])
        '''

    # build tensorflow CNN
    sess = tf.InteractiveSession()
    # input
    x = tf.placeholder(tf.float32, [None,height*width*depth],name = 'x-input')
    y_ = tf.placeholder(tf.float32, [None,nlabels],name = 'y-input')

    x_image = tf.reshape(x, [-1, height, width, depth, 1])

    ## First Convolutional Layer
    W_conv1 = weight_variable([5, 5, 5, 1, 7],name= 'W_conv1')
    b_conv1 = bias_variable([7],name = 'b_conv1')
    h_conv1 = tf.nn.relu(conv3d(x_image, W_conv1, [1,1,1], 'VALID') + b_conv1)
    print(h_conv1.get_shape)

    h_pool1 = max_pool_3D(h_conv1, [4,4,4], [4,4,4], 'VALID')
    print(h_pool1.get_shape)

    ## Second Convolutional Layer
    W_conv2 = weight_variable([5, 5, 5, 7, 17],name= 'W_conv2')
    b_conv2 = bias_variable([17],name = 'b_conv2')
    h_conv2 = tf.nn.relu(conv3d(h_pool1, W_conv2, [1,1,1], 'VALID') + b_conv2)
    print(h_conv2.get_shape)

    h_pool2 = max_pool_3D(h_conv2, [4,4,4], [4,4,4], 'VALID')
    print(h_pool2.get_shape)

    # ## Third Convolutional Layer
    # W_conv3 = weight_variable([5, 5, 5, 17, 7],name= 'W_conv3')
    # b_conv3 = bias_variable([7],name = 'b_conv3')
    # h_conv3 = tf.nn.relu(conv3d(h_pool2, W_conv3, [1,1,1], 'VALID') + b_conv3)
    # print(h_conv3.get_shape)

    # h_pool3 = max_pool_3D(h_conv3, [4,4,4], [4,4,4], 'VALID')
    # print(h_pool3.get_shape)

    ## Fully-connected layer
    W_fc1 = weight_variable([2*2*1*17,100], name= 'W_fc1')
    b_fc1 = bias_variable([100], name= 'b_fc1')

    h_pool3_flat = tf.reshape(h_pool2, [-1, 2*2*1*17])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    print(h_fc1_drop.get_shape)

    ## Readout Layer
    W_fc2 = weight_variable([100, nlabels], name= 'W_out') # [100, 3]
    b_fc2 = bias_variable([nlabels], name= 'b_out') # [3]

    y_conv = tf.matmul(h_fc1, W_fc2) + b_fc2
    print(y_conv.get_shape)

    ## Train and Evaluate the Model
    # set up for optimization (optimizer:ADAM)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)  # 1e-4
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    sess.run(tf.global_variables_initializer())
    #hxf 4/29/2018 preparing for saving the mode
    path, filename = os.path.split(result_saved_root) #refer to https://stackoverflow.com/questions/10507298/splitting-path-strings-into-drive-path-and-file-name-parts
    tempFilename=os.path.splitext(filename) #refer to https://stackoverflow.com/questions/678236/how-to-get-the-filename-without-the-extension-from-a-path-in-python
    sModelPath = path + "/" + tempFilename[0] #without extension

    # train
    for step in range(num_steps):
        print("step: "+str(step))
        begin, end, batch_x, batch_y = get_data_batch(train_X, train_y, batchsize, width, height, depth)
        # Run optimization op (backprop)
        sess.run(train_step, feed_dict={x: batch_x, y_: batch_y, keep_prob: 0.5})
        #if step % 8 == 0:
        if (step == 0) or (step % 8 == 0): #testing
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cross_entropy, accuracy], feed_dict={x: batch_x, y_: batch_y, keep_prob: 0.5})
            #Calculate validation loss and accuracy
            #refer to ratio_0.35_lr_0.0015.py at E:\hxf\Faculty\ColumbiaUniversity\dataprocess\multiModal_MachineLearning\download\Python\myCode\DTIQC\tensorflow
            startTime = time.time()
            #evaluating the model using validation dataset, hxf 4/29/2018
            if method == 3: #because the testing dataset may be too large, so do it batch
                val_accuracy = calculateAccuracy(accuracy, y_conv, x, y_, val_X, val_y, width, height,depth, keep_prob, batch=32)
            else: #method 1 &2 only 1 or 2 runs data, so it's ok to run it on the whole testing dataset
                val_accuracy=sess.run(accuracy, feed_dict={x: val_X.reshape(val_X.shape[0], width*height*depth), y_: val_y.T, keep_prob: 1.0})
            #print("Testing accuracy:", val_accuracy)
            average_time = (time.time() - startTime) / 1000
            '''
            with open(result_saved_root, 'a') as f:
                print("Step " + str(step) + ", Begin: " + str(begin) + ", end: " + str(end) + ", Minibatch Loss= " +
                    "{:.4f}".format(loss) + ", Training accuracy= " + "{:.3f}".format(acc),", Validation accuracy= " + "{:.3f}".format(val_accuracy),", average validation time = " + "{:.5f}".format(average_time), file=f)
            '''
            #dxz 8/9/2018 add totalTrainSize
            with open(result_saved_root, 'a') as f:
                print("Step " + str(step) + ", Begin: " + str(begin) + ", end: " + str(end) + ', totalTrainSize= ' + str(train_X.shape[0]) + ", Minibatch Loss= " +
                    "{:.4f}".format(loss) + ", Training accuracy= " + "{:.3f}".format(acc),", Validation accuracy= " + "{:.3f}".format(val_accuracy),", average validation time = " + "{:.5f}".format(average_time), file=f)

            '''
            with open(result_saved_root, 'a') as f:
                print("Step " + str(step) + ", Begin: " + str(begin) + ", end: " + str(end) + ", Minibatch Loss= " +
                    "{:.4f}".format(loss) + ", Training accuracy= " + "{:.3f}".format(acc), file=f)
            '''

            #based  on  https: // www.researchgate.net / post / How_do_I_know_when_to_stop_training_a_neural_network
            #see https://www.youtube.com/watch?v=S4ZUwgesjS8 to prevent overfitting.
            #You need to divide the data into a training data set and  a validation data set. You train the nnet on the training data set only. The network calculates the errors on the training data set and the validtion data set. Stop training when the validation error is the minimum. This means that the nnet can generalise to unseen data. If you stop training when the training error is minimum then you will have over fitted and the nnet cannot generalise to unseen data. The validation error is usually bigger than the training error in most cases. This process is called cross validation. There are more complicated methods to do cross validation. Cross validation does not apply just to nnets, but is a way of selecting the best model ( which may be a nnet) that produces the best generalisation to unseen data.
            # refer to ratio_0.35_lr_0.0015.py at E:\hxf\Faculty\ColumbiaUniversity\dataprocess\multiModal_MachineLearning\download\Python\myCode\DTIQC\tensorflow
            #if acc>=0.75 and val_accuracy>=0.75: #you can set it up as an input threshold
            if acc>=0.55 and val_accuracy>=0.55: #for testing purpose, in reality, need a criteria
                startTime = time.time()
                if method == 3:  # because the testing dataset may be too large, so do it batch
                    test_accuracy = calculateAccuracy(accuracy, y_conv, x, y_, test_X, test_y, width, height, depth,keep_prob, batch=32)
                else:  # method 1 &2 only 1 or 2 runs data, so it's ok to run it on the whole testing dataset
                    test_accuracy = sess.run(accuracy,feed_dict={x: test_X.reshape(test_X.shape[0], width * height * depth),y_: test_y.T, keep_prob: 1.0})
                average_time = (time.time() - startTime) / 1000
                print("Testing accuracy:", test_accuracy)
                with open(result_saved_root, 'a') as f:
                    print("......Testing result using Step" + str(step) + " model......", file=f)
                    print("...Testing accuracy:" + "{:.3f}".format(test_accuracy) + ", average testing time = " + "{:.5f}".format(average_time), file=f)
                #save the model
                try:
                    oTrainSaver
                except NameError:
                    oTrainSaver=tf.train.Saver()
                save_path = sModelPath + '_' + str(step)+'.ckpt'
                oTrainSaver.save(sess, save_path)  # filename ends with .ckpt
                #save_path = model_name+'_'+str(step)
                #if not os.path.isabs(save_path):
                #	save_path = model_saved+save_path
                #oTrainSaver.save(sess, save_path)
    print("Optimization Finished!")
    with open(result_saved_root, 'a') as f:
        print("Optimization Finished!", file=f)
    #hxf 4/28/2018, save last epoch's model
    #refer to https: // www.tensorflow.org / programmers_guide / saved_model
    #https://gist.github.com/omimo/5d393ed5b64d2ca0c591e4da04af6009
    #https://stackoverflow.com/questions/38000180/save-tensorflow-model-to-file
    try:
        oTrainSaver
    except NameError:
        oTrainSaver = tf.train.Saver()
    save_path = sModelPath + '_lastEpoch'+'.ckpt'
    oTrainSaver.save(sess, save_path)  # filename ends with .ckpt
    #oSaveSess = sess
    #oTrainSaver.save(oSaveSess, save_path)  # filename ends with .ckpt
    print("Trained model was saved!")
    with open(result_saved_root, 'a') as f:
        print("Trained model was saved!", file=f)

    startTime = time.time()
    #evaluating the model using validation dataset, hxf 4/29/2018
    if method == 3: #because the testing dataset may be too large, so do it batch
        test_accuracy = calculateAccuracy(accuracy, y_conv, x, y_, test_X, test_y, width, height,depth, keep_prob, batch=32)
    else: #method 1 &2 only 1 or 2 runs data, so it's ok to run it on the whole testing dataset
        test_accuracy=sess.run(accuracy, feed_dict={x: test_X.reshape(test_X.shape[0], width*height*depth), y_: test_y.T, keep_prob: 1.0})
    average_time = (time.time() - startTime) / 1000
    print("......Testing accuracy using the last epotch training model:", test_accuracy)
    #save the val_accuracy to a .txt file
    with open(result_saved_root, 'a') as f:
        print(".......Testing result using the last epotch training model......", file=f)
        print("...Testing accuracy:" + "{:.3f}".format(test_accuracy)+", average testing time = " + "{:.5f}".format(average_time), file=f)

    '''
    #original from Sikai
    if method == 3:
        val_accuracy = calculateAccuracy(accuracy, y_conv, x, y_, val_X, val_y, width, height,depth, keep_prob, batch=32)
        print("Testing Accuracy:", val_accuracy)
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: val_X, 
        y_: val_y, 
        keep_prob: 1.0}))
    '''

In [32]:
### Help.py
#! /cluster_NAS/scratch1/xiaofu/local/bin/python3
# -*- coding: utf-8 -*-

import tensorflow as tf
import numpy as np
import scipy.io
import random

batch_index = 0

def one_hot_matrix(labels, C):
    """
    Creates a matrix where the i-th row corresponds to the ith class number and the jth column
                     corresponds to the jth training example. So if example j had a label i. Then entry (i,j) 
                     will be 1. 
                     
    Arguments:
    labels -- vector containing the labels 
    C -- number of classes, the depth of the one hot dimension
    
    Returns: 
    one_hot -- one hot matrix
    """    
    # Create a tf.constant equal to C (depth), name it 'C'.
    C = tf.constant(C, name = 'C')
    
    one_hot_matrix = tf.one_hot(labels, depth=C, axis = 0)
    
    sess = tf.Session()
    one_hot = sess.run(one_hot_matrix)
    sess.close()
       
    return one_hot

def cost(logits, labels):
    # Create the placeholders for "logits" (z) and "labels" (y)
    y = tf.placeholder(tf.float32, name = 'labels')
    
    # Use the loss function
    cost = tf.nn.sigmoid_cross_entropy_with_logits(logits=z, labels=y)
    
    # Create a session
    sess = tf.Session()
    # Run the session
    cost = sess.run(cost, feed_dict={z:logits,y:labels})
    sess.close()

    return cost

# GRADED FUNCTION: compute_cost 

def compute_cost(Z3, Y):
    """
    Computes the cost
    
    Arguments:
    Z3 -- output of forward propagation (output of the last LINEAR unit), of shape (6, number of examples)
    Y -- "true" labels vector placeholder, same shape as Z3
    
    Returns:
    cost - Tensor of the cost function
    """
    logits = tf.transpose(Z3)
    labels = tf.transpose(Y)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    
    return cost

def get_data_batch(data_x, data_y, batch_size, width, height, depth):
    global batch_index
    max_ = data_y.shape[1]
    begin =batch_index
    end = batch_index + batch_size
    
    if end >= max_:
        end = max_
        batch_index = -batch_size
    y_data = data_y[:, begin:end].T
    fetch_data_x= data_x[begin:end,:,:,:]
        #resized_image = tf.image.resize_images(images=fetch_data_x, size=(width), method=1)
        
        #image = sess.run(resized_image)  # (256,256)
        #x_data = np.append(x_data, np.asarray(image, dtype='float32')) # (image.data, dtype='float32')

    batch_index += batch_size
    x_data_= fetch_data_x.reshape(end-begin, width*height*depth)
    
    return begin, end, x_data_, y_data

# Tensorflow part, hxf 6/7/2018, added ,trainable=True
def weight_variable(shape,name,trainable=True):
    initial = tf.truncated_normal(shape,stddev=0.1)*0.01
    return tf.Variable(initial,name = name,trainable=trainable)

def bias_variable(shape,name,trainable=True):
    initial = tf.constant(0.1,shape = shape)
    return tf.Variable(initial, name = name,trainable=trainable)
"""
def weight_variable(shape,name):
    initial = tf.truncated_normal(shape,stddev=0.1)*0.01
    return tf.Variable(initial,name = name)

def bias_variable(shape,name):
    initial = tf.constant(0.1,shape = shape)
    return tf.Variable(initial, name = name)
"""

# con layer
def conv3d(x,W, strides, paddingtype):
    return tf.nn.conv3d(x, W, strides=[1,strides[0],strides[1],strides[2],1], padding = paddingtype)

# pool
def max_pool_3D(x,poolsize, strides, paddingtype):
    return tf.nn.max_pool3d(x,ksize=[1,poolsize[0],poolsize[1],poolsize[2],1],strides=[1,strides[0],strides[1],strides[2],1],padding=paddingtype)


def calculateAccuracy(accuracy, y_conv, x, y_, validation_x, validation_y, width, height,depth, keep_prob, batch=100):
    """
    Since each tensor has memory limited, just separate the whole test set into some small parts
    """
    nums = validation_y.shape[1]
    predict_list = []
    print(nums)
    total_accuracy = 0
    data_x = validation_x.reshape(nums,width,height,depth)
    sess = tf.get_default_session()
    cal_list = range(0, nums, batch)
    end_ = cal_list[-1]
    for i in cal_list[:-1]:
        batch_x, batch_y = data_x[i:i+batch,:,:,:], validation_y[:,i:i+batch]
        x_data= batch_x.reshape(batch, width*height*depth)
        #8/19/2018 dxz   bug fixed   batch_y should do a transpose here
        validation_dic_feed = {x: x_data, y_: batch_y.T,keep_prob:1}
        #validation_dic_feed = {x: x_data, y_: batch_y,keep_prob:1}
        #validation_dic_feed.update(dict((zip(dropout_placehold_store,[1]*len(dropout_placehold_store)))))
        accu = sess.run(accuracy, feed_dict =validation_dic_feed)
        predict = sess.run(tf.argmax(y_conv,1), feed_dict =validation_dic_feed)
        predict_list.extend(predict)

        total_accuracy += (accu*batch)

    batch_x, batch_y = data_x[end_:,:,:,:], validation_y[:,end_:]
    #8/19/2018 dxz  bug fixed
    x_data= batch_x.reshape(nums-end_, width*height*depth)
    validation_dic_feed = {x: x_data, y_: batch_y.T, keep_prob:1}
    #x_data= batch_x.reshape(nums-end_, width*height)
    #validation_dic_feed = {x: x_data, y_: batch_y, keep_prob:1}
    #validation_dic_feed.update(dict((zip(dropout_placehold_store,[1]*len(dropout_placehold_store)))))
    accu = sess.run(accuracy, feed_dict =validation_dic_feed)
    predict = sess.run(tf.argmax(y_conv,1), feed_dict =validation_dic_feed)
    predict_list.extend(predict)
    total_accuracy += (accu*(nums-end_))

    return total_accuracy / nums

#def data_preprocessing(root, C=3, get_all='False'):
#hxf 4/30/2018 added testing set, but still doesn't work for C=2
def data_preprocessing(root, get_all='False', get_valset='True',train_split =0.6,val_split=0.2):
    """
    seperate data into train and validation part

    Arguments
        root -- root of mat file
        get_all='True' -- without split data, return all data and label(one-hot) 
        get_valset='True' -- with split data, return train, test, val data 
        get_valset='False' -- train_split+val_split 
        

    Returns
            train_X, train_y, val_X, val_y #get_valset='False'
            train_X, train_y, val_X, val_y, test_X, test_y #for get_valset='True'

    """
    data = scipy.io.loadmat(root)  #300, 60
    raw_data = np.concatenate((data['I'], data['I_test']), axis=1)
    raw_label = np.concatenate((data['labels'], data['labels_test']), axis=0)

    train_label_list = [element[0] for _,element in enumerate(raw_label.tolist())]
    # count the number of different labels
    C = len(set(train_label_list))

    data_ = np.array([element for _,element in enumerate(raw_data[0,:])])

    train_index_var = []
    val_index_var = []
    test_index_var = []
    for NumLabel in range(C):
        locals()['label_'+str(NumLabel)] = [index for index,element in enumerate(train_label_list) if element==NumLabel]
        locals()['random_permu_'+str(NumLabel)] = np.random.permutation(len(locals()['label_'+str(NumLabel)]))
    #label_0 = [index for index,element in enumerate(train_label_list) if element==0]
    #label_1 = [index for index,element in enumerate(train_label_list) if element==1]
    #label_2 = [index for index,element in enumerate(train_label_list) if element==2]
        if get_valset == 'True': #hxf 4/30/2018 added testing set, i.e., 60% traing, 20% validation, 20% testing
            locals()['train_index_'+str(NumLabel)] = np.array(locals()['label_'+str(NumLabel)])[locals()['random_permu_'+str(NumLabel)][0:int(np.ceil(len(locals()['label_'+str(NumLabel)]) * train_split))].tolist()]
            locals()['val_index_'+str(NumLabel)] = np.array(locals()['label_'+str(NumLabel)])[locals()['random_permu_'+str(NumLabel)][int(np.ceil(len(locals()['label_'+str(NumLabel)]) * train_split)):int(np.ceil(len(locals()['label_'+str(NumLabel)]) * (train_split+val_split)))].tolist()]
            val_index_var.append(locals()['val_index_'+str(NumLabel)])
        else: #i.e., 80% training 20% testing
            locals()['train_index_'+str(NumLabel)] = np.array(locals()['label_'+str(NumLabel)])[locals()['random_permu_'+str(NumLabel)][0:int(np.ceil(len(locals()['label_'+str(NumLabel)]) * (train_split+val_split)))].tolist()]
        locals()['test_index_'+str(NumLabel)] = np.array(locals()['label_'+str(NumLabel)])[locals()['random_permu_'+str(NumLabel)][int(np.ceil(len(locals()['label_'+str(NumLabel)]) * (train_split+val_split))):].tolist()]
        
        '''BUG HERE
        train_index_var.append(locals()['label_'+str(NumLabel)])  ####bugs might comes from here
        #val_index_var.append(locals()['random_permu_'+str(NumLabel)])
        #test_index_var.append(locals()['test_index_'+str(NumLabel)])
        '''

        train_index_var.append(locals()['train_index_'+str(NumLabel)])  ####xzd 7/2/2018 
        test_index_var.append(locals()['test_index_'+str(NumLabel)])
        
    if get_all == 'True':
        random_index = np.random.permutation(len(train_label_list))
        data_X = data_[random_index,:,:,:]
        data_y = one_hot_matrix(np.array(train_label_list)[random_index], C)

        assert(data_X.shape == data_.shape)
        assert(data_y.shape[1] == len(train_label_list))

        return data_X, data_y

    #random_permu_0 = np.random.permutation(len(label_0))
    #random_permu_1 = np.random.permutation(len(label_1))
    #random_permu_2 = np.random.permutation(len(label_2))

    #if get_valset == 'True': #hxf 4/30/2018 added testing set, i.e., 60% traing, 20% validation, 20% testing
    #    train_index_0 = np.array(label_0)[random_permu_0[0:int(np.ceil(len(label_0) * 0.6))].tolist()]
    #    val_index_0 = np.array(label_0)[random_permu_0[int(np.ceil(len(label_0) * 0.6)):int(np.ceil(len(label_0) * 0.8))].tolist()]
    #    test_index_0 = np.array(label_0)[random_permu_0[int(np.ceil(len(label_0) * 0.8)):].tolist()]
    #
    #    train_index_1 = np.array(label_1)[random_permu_1[0:int(np.ceil(len(label_1) * 0.6))].tolist()]
    #    val_index_1 = np.array(label_1)[random_permu_1[int(np.ceil(len(label_1) * 0.6)):int(np.ceil(len(label_1) * 0.8))].tolist()]
    #    test_index_1 = np.array(label_1)[random_permu_1[int(np.ceil(len(label_1) * 0.8)):].tolist()]
    #
    #    train_index_2 = np.array(label_2)[random_permu_2[0:int(np.ceil(len(label_2) * 0.6))].tolist()]
    #    val_index_2 = np.array(label_2)[random_permu_2[int(np.ceil(len(label_2) * 0.6)):int(np.ceil(len(label_2) * 0.8))].tolist()]
    #    test_index_2 = np.array(label_2)[random_permu_2[int(np.ceil(len(label_2) * 0.8)):].tolist()]
    #else: #i.e., 80% training 20% validation
    #    train_index_0 = np.array(label_0)[random_permu_0[0:int(np.ceil(len(label_0) * 0.8))].tolist()]
    #    val_index_0 = np.array(label_0)[random_permu_0[int(np.ceil(len(label_0) * 0.8)):].tolist()]
    #    train_index_1 = np.array(label_1)[random_permu_1[0:int(np.ceil(len(label_1) * 0.8))].tolist()]
    #    val_index_1 = np.array(label_1)[random_permu_1[int(np.ceil(len(label_1) * 0.8)):].tolist()]
    #    train_index_2 = np.array(label_2)[random_permu_2[0:int(np.ceil(len(label_2) * 0.8))].tolist()]
    #    val_index_2 = np.array(label_2)[random_permu_2[int(np.ceil(len(label_2) * 0.8)):].tolist()]

    # train index
    train_index = np.concatenate(tuple(train_index_var))
    random.shuffle(train_index)
    # train X
    train_X = data_[train_index, :, :, :]
    # train y
    train_y = one_hot_matrix(np.array(train_label_list)[train_index], C)
    # print(train_X.shape[0])
    # print(train_y.shape[0])
    
    #testing index
    test_index = np.concatenate(tuple(test_index_var))
    #random.shuffle(test_index)
    # testing X
    test_X = data_[test_index, :, :, :]
    # testing y
    test_y = one_hot_matrix(np.array(train_label_list)[test_index], C)
    # print(test_X.shape[0])
    # print(test_X.shape[0])



    if get_valset == 'True': #hxf 4/30/2018 added testing set, i.e., 60% traing, 20% validation, 20% testing

        #validation index
        val_index = np.concatenate(tuple(val_index_var))
        random.shuffle(val_index)
        # validation X
        val_X = data_[val_index, :, :, :]
        # validation y
        val_y = one_hot_matrix(np.array(train_label_list)[val_index], C)
        # print(val_X.shape[0])
        # print(val_y.shape[0])
        assert (test_X.shape[0] == test_y.shape[1])

        assert (train_X.shape[0] == train_y.shape[1])
        assert (val_X.shape[0] == val_y.shape[1])
        assert ((train_X.shape[0] + val_X.shape[0] + test_X.shape[0]) == len(train_label_list))
        return train_X, train_y, val_X, val_y, test_X, test_y
    else:
        assert (train_X.shape[0] == train_y.shape[1])
        assert ((train_X.shape[0] + test_X.shape[0]) == len(train_label_list))
        return train_X, train_y, test_X, test_y

    '''	
    train_index_0 = np.array(label_0)[random_permu_0[0:int(np.ceil(len(label_0)*0.8))].tolist()]
    test_index_0 = np.array(label_0)[random_permu_0[int(np.ceil(len(label_0)*0.8)):].tolist()]
    train_index_1 = np.array(label_1)[random_permu_1[0:int(np.ceil(len(label_1)*0.8))].tolist()]
    test_index_1 = np.array(label_1)[random_permu_1[int(np.ceil(len(label_1)*0.8)):].tolist()]
    train_index_2 = np.array(label_2)[random_permu_2[0:int(np.ceil(len(label_2)*0.8))].tolist()]
    test_index_2 = np.array(label_2)[random_permu_2[int(np.ceil(len(label_2)*0.8)):].tolist()]

    train_index = np.concatenate((train_index_0,train_index_1,train_index_2))
    random.shuffle(train_index)

    # train X
    train_X = data_[train_index,:,:,:]

    val_index = np.concatenate((test_index_0,test_index_1,test_index_2))
    random.shuffle(val_index)

    # validation X
    val_X = data_[val_index,:,:]

    # train y
    train_y = one_hot_matrix(np.array(train_label_list)[train_index], C)

    # validation y
    val_y = one_hot_matrix(np.array(train_label_list)[val_index], C)
    #print(train_X.shape[0])
    #print(train_y.shape[0])

    assert(train_X.shape[0]==train_y.shape[1])
    assert(val_X.shape[0]==val_y.shape[1])
    assert((train_X.shape[0]+val_X.shape[0]) == len(train_label_list))

    return train_X, train_y, val_X, val_y
    '''

if __name__ == '__main__':
    #root='D:\\dataprocess\\shared\\Python\\BrainState\\MRIcluster\\data\\rtfMRI\\wholeBrain\\E14163_run7_8_16_3D_wholeBrain_BlankVSNegative.mat'
    root = '../data/E14163_run7_3D_wholeBrain_allConds.mat'
    #root = r'D:\dataprocess\shared\Python\BrainState\MRIcluster\data\rtfMRI\wholeBrain\E14163_run7_8_16_3D_wholeBrain_allConds.mat'
    print(root)
    train_X, train_y, val_X, val_y, test_X, test_y = data_preprocessing(root)
    #train_X, train_y, val_X, val_y, test_X, test_y = data_preprocessing('/cluster_NAS/scratch1/xiaofu/data/rtfMRI/exclude4s/half/wholeBrain/E14178_run11_3D_wholeBrain_allConds.mat')
    print(train_X.shape)
    print(train_y.shape)
    print(val_X.shape)
    print(val_y.shape)
    print(test_X.shape)
    print(test_y.shape)






../data/E14163_run7_3D_wholeBrain_allConds.mat


FileNotFoundError: [Errno 2] No such file or directory: '../data/E14163_run7_3D_wholeBrain_allConds.mat'

In [33]:
if __name__ == '__main__':
    # train with method 1
    #model(width, height, depth, nlabels, num_steps, folder_root, file_name, batchsize=16, method=1)
    # train with method2
    file_pair = {"train":'../data/E14163_run7_3D_wholeBrain_allConds.mat',"val":'../data/E14163_run8_3D_wholeBrain_allConds.mat',"test": 'E14163_run16_3D_wholeBrain_allConds.mat'}
    model(width, height, depth, nlabels, num_steps, folder_root, file_name, batchsize=16, learning_rate=0.01,method=3)
    # train with method3
    # file_pair = {"train":'E13260_run8_3D_wholeBrain_allConds.mat',"val":'E13260_run9_3D_wholeBrain_allConds.mat'}
    #model(width, height, depth, nlabels, num_steps, folder_root, file_pair, batchsize=16, method=3)


/Users/gimdong-geon/python3_cooking/lib/python3.7/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


<bound method Tensor.get_shape of <tf.Tensor 'Relu_12:0' shape=(?, 60, 60, 40, 7) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'MaxPool3D_8:0' shape=(?, 15, 15, 10, 7) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Relu_13:0' shape=(?, 11, 11, 6, 17) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'MaxPool3D_9:0' shape=(?, 2, 2, 1, 17) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'dropout_4/mul_1:0' shape=(?, 100) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'add_19:0' shape=(?, 3) dtype=float32>>
step: 0
0


IndexError: range object index out of range